In [1]:
# automated ref adding for lineage-note.txt, written by Xu Zou


import bs4
from bs4 import BeautifulSoup
from urllib import request
import os
import json
# Use AngieHinrich's better query code for milestones

# Build milestone_list from API query that lists all milestones in a repo.
milestone_list = []
milestone_already=[]
milestone_title_dict={}
listd=os.listdir()

reassign_milestone=True
if not("autoref" in listd):
    os.mkdir("autoref")
lista=os.listdir("autoref")

if not(reassign_milestone):
    if ("milestone.txt" in lista):
        w=open("autoref/milestone.txt",'r')
        milestones=w.readlines()
        for line in milestones:
            linsp=line.split()
            if len(linsp)>2:
                issue=linsp[0]
                name=linsp[1]
                milestone_list.append([linsp[0],linsp[1],linsp[2]])
                if linsp[1]!='withdrawn':
                    milestone_title_dict[linsp[1]]=[linsp[2],linsp[0]]
                milestone_already.append(issue)
        w.close()

# refresh token

w=open("autoref/milestone.txt",'w')
for item in milestone_list:
    print(item[0],item[1],item[2],file=w)
item_count=0
for repo in ['cov-lineages/pango-designation', 'sars-cov-2-variants/lineage-proposals']:
    if repo == 'sars-cov-2-variants/lineage-proposals':
        prefix = 'sars#'
    else:
        prefix = ''
    # Request pages starting with 1, until we get an empty result list.
    page = 1
    while 1:
        url = f'https://api.github.com/repos/{repo}/milestones?per_page=100&page={page}'
        res_text = request.urlopen(url).read().decode('utf-8')
        res_list = json.loads(res_text)
        
        if len(res_list) == 0:
            break
        # Each element of this page's result list includes the milestone number and title.  Parse out lineage names from title.
        
        for milestone_info in res_list:
            #print(milestone_info)
            
            milestone = milestone_info['number']
            milestone=prefix+str(milestone)
            num_issues=milestone_info['open_issues']+milestone_info['closed_issues']
            if (milestone not in milestone_already) and (num_issues>0):
                
                try:
                    title = milestone_info['title']
                    title_words = title.replace('. ','.').replace(',', ' ').split()
                    title = title.replace(' ', '+')
                    
                    for item in title_words:
                        milestone_list.append([milestone, item, title])
                        if item!='withdrawn':
                            # only add milestones to those without already assigned
                            if not(item in milestone_title_dict):
                                milestone_title_dict[item]=[title,milestone]
                            # update sars-cov-2 to main branch
                            if ("sars" in milestone_title_dict[item][1] and not('sars' in milestone)):
                                milestone_title_dict[item]=[title,milestone]
                        item_count+=1
                        print(milestone, item,title,file=w)
                        if item_count%10==0:
                            print("processed ",item_count,"milestones")
                except Exception as e:
                    print(milestone, e)
        page += 1
w.close()


processed  10 milestones
processed  20 milestones
processed  30 milestones
processed  40 milestones
processed  50 milestones
processed  60 milestones
processed  70 milestones
processed  80 milestones
processed  90 milestones
processed  100 milestones
processed  110 milestones
processed  120 milestones
processed  130 milestones
processed  140 milestones
processed  150 milestones
processed  160 milestones
processed  170 milestones
processed  180 milestones
processed  190 milestones
processed  200 milestones
processed  210 milestones
processed  220 milestones
processed  230 milestones
processed  240 milestones
processed  250 milestones
processed  260 milestones
processed  270 milestones
processed  280 milestones
processed  290 milestones
processed  300 milestones
processed  310 milestones
processed  320 milestones
processed  330 milestones
processed  340 milestones
processed  350 milestones
processed  360 milestones
processed  370 milestones
processed  380 milestones
processed  390 milest

In [3]:
import json
                
def read_milestone_issues(name):
    # or sars-cov-2 proj if it starts to add milestones
    
    # first, deal with multiple names
    #print(name)
    real_milestone=milestone_title_dict[name]
    di='cov-lineages/pango-designation'
    if 'sars' in real_milestone[1]:
        di='sars-cov-2-variants/lineage-proposals'
    actual_milestone='"'+real_milestone[0]+'"'
    url="https://github.com/"+di+"/issues?q=milestone:"+actual_milestone
    print(url)
    res=request.urlopen(url)
    rp=res.read()
    rp=rp.decode('utf-8')
    html_soup=BeautifulSoup(rp,"html")
    #print(html_soup)
    #html=html_soup.text
    target_attrs = {
        'data-target': 'react-app.embeddedData',
        'type': 'application/json'
    }

    # Find the script tag using the specified attributes
    target_script_tag = html_soup.find('script', attrs=target_attrs)

    #print("preloadedQueries:",target_script_tag.string)
    data=json.loads(target_script_tag.string)
    
    
    
    
    try:
        payload=data['payload']['preloadedQueries'][0]['result']['data']['repository']['search']['edges'][0]['node']
        
        outid=str(payload['number'])
        if 'sars' in real_milestone[1]:
            outid='sars#'+outid
        return outid
    except Exception as e:
        print(name,e)
        return '0'

w=read_milestone_issues("NL.4.1.1")
print(w)

https://github.com/sars-cov-2-variants/lineage-proposals/issues?q=milestone:"NL.4.1+NL.4.1.1"
sars#2342


In [5]:
def examine_line(line):
    # deal with issues/ situation
    if 'issues/' in line:
        linsp=line.split('issues/')
        rel=linsp[0]+'issues/'
        pos=0
        while (linsp[1][pos]>='0' and linsp[1][pos]<='9' and pos<len(linsp[1])):
            pos+=1
        rel=rel+linsp[1][:pos]+'\n'
        return rel
    if '#' in line:
        return line
    return ''
    

In [7]:
# read lineage_notes.txt


def milestone2issue(num):
    return 0
w=open("lineage_notes.txt",'r',encoding='utf-8')
lineage_notes=w.readlines()
w.close()
milestone_tobecheck=[]


changed_items=0
missing_milestones=[]
for num in range(1,len(lineage_notes)):
    line=lineage_notes[num]
    linsp=line.split()
    if (len(linsp)>=2):
        title=linsp[0]
        # for all lineages without an issue label but with a milestone
        if not('#' in line or 'issues/' in line):
            if title in milestone_title_dict:
                # this time we need to deal with the milestone 
                #print(title)
                issue_num=read_milestone_issues(title)
                if issue_num!='0':
                    if 'sars' in issue_num:
                        issue_num=issue_num.replace('sars','sars-cov-2-variants/lineage-proposals')
                    else:
                        issue_num='#'+issue_num
                        
                    missing_milestones.append([title,issue_num])
                    print(missing_milestones[-1])
        if ('lineage-proposals#' in line):
            if title in milestone_title_dict:
                if not('sars' in milestone_title_dict[title][1]):
                    issue_num=read_milestone_issues(title)
                    if issue_num!='0':
                        issue_num='#'+issue_num

                        missing_milestones.append([title,issue_num])
                        print(missing_milestones[-1])
                    
#rewrite lineage_notes.txt
w=open("autoref/missing_refs.txt",'w')
for item in missing_milestones:
    print(item[0],item[1],file=w)
w.close()
        


https://github.com/sars-cov-2-variants/lineage-proposals/issues?q=milestone:"NY.7.1.2"
['NY.7.1.2', 'sars-cov-2-variants/lineage-proposals#2874']
https://github.com/sars-cov-2-variants/lineage-proposals/issues?q=milestone:"QM.2"
['QM.2', 'sars-cov-2-variants/lineage-proposals#3051']
https://github.com/sars-cov-2-variants/lineage-proposals/issues?q=milestone:"RG.1+RG.1.1+RG.1.2"
['RG.1', 'sars-cov-2-variants/lineage-proposals#2872']
https://github.com/sars-cov-2-variants/lineage-proposals/issues?q=milestone:"RG.1+RG.1.1+RG.1.2"
['RG.1.1', 'sars-cov-2-variants/lineage-proposals#2872']
https://github.com/sars-cov-2-variants/lineage-proposals/issues?q=milestone:"RG.1+RG.1.1+RG.1.2"
['RG.1.2', 'sars-cov-2-variants/lineage-proposals#2872']
https://github.com/sars-cov-2-variants/lineage-proposals/issues?q=milestone:"NY.1+NY.1.2+NY.1.2.1+NY.3+NY.3.2+NY.4.1++NY.12+NY.14+NY.16.1+NY.17.1+NY.17.2+NY.18.1+NY.19+NY.20.1+NY.25.1+NY.26.1+NY.26.3+LP.8.1.3+LP.8.1.6+PF.1+PF.2+PF.3.1+PF.4+LP.8.1.9+NW.1+NW

In [9]:
#final rewrite of lineage_notes.txt
w=open("autoref/missing_refs.txt",'r',encoding='utf-8')
data=w.readlines()
w.close()
missing_refs={}
for line in data:
    linsp=line.split()
    missing_refs[linsp[0]]=linsp[1].replace('\n','')

w=open("lineage_notes.txt",'r',encoding='utf-8')
data=w.readlines()
w.close()
normal_count=0
sub_count=0
upgrade_count=0
for i in range(len(data)):
    linsp=data[i].split()
    
    if (len(linsp)>0) and (linsp[0] in missing_refs):
        if "，from" in data[i]:
            data[i]=data[i].split(", from")[0]
            upgrade_count+=1
        data[i]=data[i].replace('\n','')+", from "+missing_refs[linsp[0]]+'\n'
        if 'sars' in missing_refs[linsp[0]]:
            sub_count+=1
        else:
            normal_count+=1

print("Add ",normal_count-upgrade_count," references from cov-lineages/pango-designation." )
print("Upgrade ",upgrade_count," references from cov-lineages/pango-designation." )
print("Add ",sub_count," references from sars-cov-2-variants/lineage-proposals.")

w=open("lineage_notes.txt",'w',newline='\n',encoding='utf-8')
w.writelines(data)
w.close()


Add  0  references from cov-lineages/pango-designation.
Upgrade  0  references from cov-lineages/pango-designation.
Add  63  references from sars-cov-2-variants/lineage-proposals.


In [12]:
# replace all "From" with ", from" 
w=open("lineage_notes.txt",'r')
lines=w.readlines()
for i in range(len(lines)):
    lines[i]=lines[i].replace(" From",", from")
w.close()
w=open("lineage_notes.txt",'w',newline='\n')
w.writelines(lines)
w.close()
